In [1]:
import nibabel as nib
import numpy as np
from nilearn.image import new_img_like

In [2]:
import nibabel as nib
import numpy as np
from nilearn.image import new_img_like

def generate_sphere_at_mni(coords, radius, template_img_path, output_path):
    """
    Generate a spherical mask at specified MNI coordinates.

    Parameters:
        coords (tuple): MNI coordinates (x, y, z)
        radius (float): Radius of the sphere in millimeters
        template_img_path (str): Path to the template image for affine
        output_path (str): Path to save the generated spherical mask

    Returns:
        nibabel.Nifti1Image: Nifti image of the generated sphere
    """
    template_img = nib.load(template_img_path)
    affine = template_img.affine
    inv_affine = np.linalg.inv(affine)
    data_shape = template_img.shape
    mask_data = np.zeros(data_shape, dtype=np.uint8)

    # Compute voxel indices for sphere
    center_voxel = nib.affines.apply_affine(inv_affine, coords).astype(int)
    x, y, z = np.indices(data_shape)
    dist = np.sqrt((x - center_voxel[0])**2 + (y - center_voxel[1])**2 + (z - center_voxel[2])**2)
    mask_data[dist <= (radius / np.mean(np.abs(np.diag(affine)[:3])))] = 1

    # Create and save NIfTI image
    sphere_img = new_img_like(template_img, mask_data)
    sphere_img.to_filename(output_path)

    return sphere_img



In [3]:
# Example usage
sphere_img = generate_sphere_at_mni(coords=(-62, -14, 2),
                                    radius=6,
                                    template_img_path='mni152.nii.gz',
                                    output_path='AWFA_sphere.nii.gz')
